<a href="https://colab.research.google.com/github/RxnAch/DeepLearning/blob/main/Linear_regression_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from datetime import date
today = date.today()
print("Today's date:", today)

Today's date: 2021-05-30


In [ ]:
!pip install -U d2l

In [ ]:
%matplotlib inline
import random
import torch
import numpy as np
from d2l import torch as d2l

In [ ]:
#Generating the dataset
def synthetic_data(w,b,num_examples):
  X = torch.normal(0,1,(num_examples,len(w)))
  y = torch.matmul(X,w)+b #y = X.w + b
  y+= torch.normal(0,0.001,y.shape) #add some noises
  return X,y.reshape((-1,1)) 
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features , labels = synthetic_data(true_w,true_b,1000)

#Reading the dataset
def data_iter(batch_size,features,labels): #creating minibatches
  num_examples = len(features)
  indices = list(range(num_examples)) 
  random.shuffle(indices)
  for i in range(0 ,num_examples,batch_size):
    batch_indices = torch.tensor(indices[i:i+batch_size])

    yield features[batch_indices],labels[batch_indices]

batch_size = 10

#Defining the Model

def linreg(X,w,b):
  return torch.matmul(X,w)+b

#Defining the loss function
def squared_loss(y_hat,y):
  return (y_hat - y.reshape(y_hat.shape))**2 /2

#Defining the Optimization Algorithm
def sgd(params,lr,batch_size):
  with torch.no_grad():
    for param in params:
      param -= lr * param.grad / batch_size  
      param.grad.zero_()
      
#initializing the model parameters
w = torch.normal(0,0.01,size = (2,1),requires_grad = True)
b = torch.zeros(1,requires_grad = True)

#Training
lr = 0.03
num_epochs = 3
model = linreg
loss = squared_loss
for epoch in range(num_epochs):
  for X,y in data_iter(batch_size,features,labels):
    l = loss(model(X,w,b),y)
    l.sum().backward() #compute gradient on 'l' w.r.t ['w','b']
    sgd([w,b],lr,batch_size) #update parameters using their gradient
  with torch.no_grad():
    train_l = loss(model(features,w,b),labels)
  print(f'epoch {epoch+1},loss {float(train_l.mean()):f}')
  

epoch 1,loss 0.037850
epoch 2,loss 0.000090
epoch 3,loss 0.000001
